# USER INPUT

**Paste the Excel Line in the Cell Below!**

In [1]:
params = {
"event_id": "nc73821036",
"geography": "tract",
"intensity_metric": "min",
"utility_loss_rate": 0.20
}

# CODE BASE




In [2]:
EVENTID = params["event_id"]

In [3]:
import os

In [9]:
# Mount the Google Drive
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [5]:
# Setting System to Import Packages
import sys
sys.path.append('/content/drive/MyDrive/capstone/WorkingScripts')
parent_dir = '/content/drive/MyDrive/capstone/'

In [4]:
parent_dir = os.path.dirname(os.getcwd())

# SKIP


In [18]:
# Install Required Packages
import environment_setting as es # Replace my_module with the actual name of your module

es.setting_env(parent_dir)

Environment is set up successfully.


In [19]:
# Running o1
import o1_getshakemap as getshakemap

feed_url = getshakemap.FEEDURL.format(EVENTID)
jdict = getshakemap.fetch_earthquake_data(feed_url=feed_url)

event = getshakemap.retrieve_event_data(jdict)
getshakemap.download_and_extract_shakemap(event)

nc73821036: ShakeMap already downloaded.


'/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/ShakeMaps/nc73821036'

In [20]:
# Running o2 census intersect
import o2_census_intersect as census

event_dir = os.path.join(parent_dir, 'ShakeMaps', EVENTID)

census.shakemap_into_census_geo(event_dir, parent_dir)

# Update with the actual path
GPKG_PATH = os.path.join(event_dir, "eqmodel_outputs.gpkg")


mi: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/ShakeMaps/nc73821036/mi.shp
pgv: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/ShakeMaps/nc73821036/pgv.shp
pga: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/ShakeMaps/nc73821036/pga.shp
Census tracts shapefile already exists /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/Data/merged_shapefile/Nationwide_Tracts.gpkg
---GeoPackage already exists: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/ShakeMaps/nc73821036/eqmodel_outputs.gpkg
Saved shakemap_tractclip_mmi (clipped ShakeMap to tracts) to /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal

# RUN

In [5]:
# Running o3 clip building
import o3_clip_eventdata_buildingstocks as bs
bs.building_clip_analysis(parent_dir, EVENTID)

1. Reading event data for event ID: nc73821036
2. Reading building count data...
3. Reading building stock data...
Building stock data exists at /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/Data/building_stock_data/Building_Percentages_Per_Tract_ALLSTATES.csv
4. Merging building count and building stock data...
5. Merging event data with building data...
Saving final output to CSV...


In [6]:
# Run o4
import o4_TractLevel_DamageAssessmentModel as o4
o4_output = o4.main(parent_dir, EVENTID)

In [7]:
o4_output['GEOID'] = o4_output['GEOID'].astype(str)

In [8]:
o4_output.head()

,GEOID,max_intensity,min_intensity,mean_intensity,geometry,Total_Num_Building,Total_Num_Building_Slight,Total_Num_Building_Moderate,Total_Num_Building_Extensive,Total_Num_Building_Complete
0,6001400100,0.01,0.01,0.01,MULTIPOLYGON (((-122.24692041680534 37.8854393...,1298.0,0.000384,1.397666e-10,4.064599e-18,4.480291e-27
1,6001400200,0.01,0.01,0.01,MULTIPOLYGON (((-122.25792355695359 37.8426073...,696.0,0.000311,3.601871e-10,4.727889e-17,7.762607e-26
2,6001400300,0.01,0.01,0.01,MULTIPOLYGON (((-122.26563158321628 37.8376373...,1629.0,0.000826,9.106309e-10,1.002471e-16,1.615245e-25
3,6001400400,0.01,0.01,0.01,MULTIPOLYGON (((-122.2618265652303 37.84162437...,1228.0,0.000577,7.440906e-10,9.536990e-17,1.561482e-25
4,6001400500,0.01,0.01,0.01,MULTIPOLYGON (((-122.26950755249248 37.8485833...,1018.0,0.000559,7.481115e-10,9.537085e-17,1.561482e-25


In [9]:
import o5_bhi
import os
parent_dir = os.path.dirname(os.getcwd())
df = o5_bhi.process_bhi(parent_dir, o4_output)
df.head()

        GEO_ID                                           NAME P1_001N
0  06001400100  Census Tract 4001, Alameda County, California    3038
1  06001400200  Census Tract 4002, Alameda County, California    2001
2  06001400300  Census Tract 4003, Alameda County, California    5504
3  06001400400  Census Tract 4004, Alameda County, California    4112
4  06001400500  Census Tract 4005, Alameda County, California    3644
        GEOID  max_intensity  min_intensity  mean_intensity  \
0  6001400100           0.01           0.01            0.01   
1  6001400200           0.01           0.01            0.01   
2  6001400300           0.01           0.01            0.01   
3  6001400400           0.01           0.01            0.01   
4  6001400500           0.01           0.01            0.01   

                                            geometry  Total_Num_Building  \
0  MULTIPOLYGON (((-122.24692041680534 37.8854393...              1298.0   
1  MULTIPOLYGON (((-122.25792355695359 37.8426073

,GEOID,max_intensity,Total_Num_Building,Total_Num_Building_Slight,Total_Num_Building_Moderate,Total_Num_Building_Extensive,Total_Num_Building_Complete,risk_level,num_FU,perc_FU_NH_low,perc_FU_NH_high,num_PU,perc_PU_NH_low,perc_PU_NH_high,num_NU,BHI_factor_low,BHI_factor_high,population,denominator
0,06023011100,0.64,2398.0,338.441448,32.818383,2.218915,1.398090e-02,low,367.481602,0.05,0,4.821398,0.05,0.1,1.189726,0.008259,0.000697,4724,2398.0
1,06023010902,0.60,3367.0,34.294926,0.275334,0.000189,2.222358e-09,low,34.534509,0.05,0,0.035841,0.05,0.1,0.000100,0.000513,0.000001,4074,3367.0
2,06023011000,0.54,2042.0,831.352281,674.346964,183.374078,8.425120e+01,low,1458.376437,0.05,0,135.193649,0.05,0.1,179.754434,0.127048,0.094649,4625,2042.0
3,06023010901,0.54,1595.0,708.161989,479.789037,100.200062,3.577549e+01,low,1147.622466,0.05,0,88.138100,0.05,0.1,88.166012,0.094015,0.060802,4725,1595.0
4,06023010800,0.54,2529.0,822.325654,122.028606,20.651857,1.016026e+00,low,933.033950,0.05,0,21.047004,0.05,0.1,11.941190,0.023585,0.005554,4910,2529.0


In [17]:
import o5_bhi

df = o5_bhi.process_bhi(parent_dir, o4_output)
df["population"] = df["population"].astype(int)
df["shelter_seeking_low"] = df["BHI_factor_low"]*df["population"]
df["shelter_seeking_high"] = df["BHI_factor_high"]*df["population"]
cols = ["GEOID", "max_intensity", "population", "Total_Num_Building", "risk_level",
        "BHI_factor_low", #"BHI_factor_high",
        "shelter_seeking_low", #"shelter_seeking_high",
        "Total_Num_Building_Slight", "Total_Num_Building_Moderate", 
            "Total_Num_Building_Extensive", "Total_Num_Building_Complete"]

# df["numerator_low"] = round(df["num_FU"]*df["perc_FU_NH_low"] + df["num_PU"]*df["perc_PU_NH_low"] + df["num_NU"],2)
# df["numerator_high"] = round(df["num_FU"]*df["perc_FU_NH_high"] + df["num_PU"]*df["perc_PU_NH_high"] + df["num_NU"],2)

df = df[cols]

df["BHI_factor_low"] = df["BHI_factor_low"].apply(lambda x: round(x,4))
# df["BHI_factor_high"] = df["BHI_factor_high"].apply(lambda x: round(x,4))
df["shelter_seeking_low"] = df["shelter_seeking_low"].apply(lambda x: round(x,4))
# df["shelter_seeking_high"] = df["shelter_seeking_high"].apply(lambda x: round(x,4))
# print(df.sort_values(by=["shelter_seeking_low"], ascending=False).head(50))

# BHI (census) = BHI_factor * census tract population --> number of people in census tract with non-habitable housing

df.to_csv(os.path.join(parent_dir, "Data/bhi_output_new.csv"), index=False)
print("saved")

453     7.291882e+01
431     5.774848e+01
451     5.738112e+01
435     5.644869e+01
434     5.620286e+01
            ...     
422     1.862354e-06
423     1.862354e-06
1027    1.222099e-06
102     3.326792e-07
382     2.674510e-08
Length: 1513, dtype: float64
453    6.759682e+00
451    4.406905e+00
432    1.376617e+00
431    1.260724e+00
435    1.166891e+00
           ...     
934    2.142948e-16
935    2.142948e-16
423    4.708399e-17
422    4.708399e-17
382    6.630280e-19
Length: 1513, dtype: float64
453    1.797544e+02
451    8.816601e+01
432    2.122940e+01
450    1.194119e+01
431    9.200845e+00
           ...     
934    4.937749e-28
933    4.937749e-28
423    1.312965e-28
422    1.312965e-28
382    9.247804e-31
Name: num_NU, Length: 1513, dtype: float64
453     2.594329e+02
451     1.499540e+02
431     6.821005e+01
435     6.436663e+01
434     6.229637e+01
            ...     
422     1.862354e-06
423     1.862354e-06
1027    1.222099e-06
102     3.326792e-07
382     2.674510e-

/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Gradschool/4_SPRING25/Capstone/github/EarthquakeDamageModel_Heinz/WorkingScripts/o5_bhi.py:65: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72) have mixed types. Specify dtype option on import or set low_memory=False.
  pop_data = pd.read_csv(census_path)
